# Load data from the AVT smart lab

This script should collect the electricity data from 11 sensors of the Technikhaus. The machines include: 
- Th01: Drehmaschine
- Th02:	Drehmaschine
- Th03:	Drehfräse
- Th04:	Fräsmaschine
- Th05:	Drahterodiermaschine
- Th06:	5-Achsen-Fräse
- Th07:	5-Achsen-Fräse
- Th08:	5-Achsen-Fräse
- Th09:	Drehmaschine
- Th10:	Water chiller
- Th11:	Flüssigkeitskühler

Selected measured values are: current, voltage, power, harmonics, etc. 

# ToDo: 
- Sollen die Daten von der IPE Datenbank oder den Sensoren kommen?
- Falls Sensoren: Welche IP-Adressen haben die?